In [52]:
import numpy as np

In [ ]:
mass_feed_kgph = 150
mol_weights = {'P': 44, 'B': 58, 'PP': 42}


z_feed = {'P': 0.5, 'PP': 0.2, 'B': 0.3} # Mole fractions of the feed

# Average molecular weight of feed
avg_MW = sum([z_feed[c] * mol_weights[c] for c in mol_weights])
mol_feed = (mass_feed_kgph * 1000) / avg_MW 


comp = {
    'T1': {'P': 0.9, 'PP': 0.08, 'B': 0.02},
    'B1': {'P': 0.2, 'PP': 0.2, 'B': 0.6},
    'T2': {'P': 0.95, 'PP': 0.03, 'B': 0.02},
    'B2': {'P': 0.05, 'PP': 0.85, 'B': 0.10}
}

# Equations:
# 1. T1 + B1 = mol_feed
# 2. T2 + B2 = B1
# 3. Propane balance: mol_feed * z_P = T1 * x_T1(P) + B1 * x_B1(P)
# 4. Propylene balance: mol_feed * z_PP = T1 * x_T1(PP) + B1 * x_B1(PP)

# Coefficients matrix A and RHS vector b
A = np.array([
    [1, 1, 0, 0],                               # T1 + B1 = F
    [0, -1, 1, 1],                              # T2 + B2 = B1
    [comp['T1']['P'], comp['B1']['P'], 0, 0],   # P balance
    [comp['T1']['PP'], comp['B1']['PP'], 0, 0]  # PP balance
])

b = np.array([
    mol_feed,
    0,
    mol_feed * z_feed['P'],
    mol_feed * z_feed['PP']
])


In [57]:
b = b.reshape(-1, 1)
# A.shape, b.shape
A, b


(array([[ 1.  ,  1.  ,  0.  ,  0.  ],
        [ 0.  , -1.  ,  1.  ,  1.  ],
        [ 0.9 ,  0.2 ,  0.  ,  0.  ],
        [ 0.08,  0.2 ,  0.  ,  0.  ]]),
 array([[3138.07531],
        [   0.     ],
        [1569.03766],
        [ 627.61506]]))

In [58]:
def LU_decomposition(a, b):
    l  = np.zeros((len(a), len(a[0])))
    n = len(a)
    c = np.concatenate((a,b), axis=1)
    for i in range(n-1):
        max_idx_in_bottom = np.argmax(c[i:, i])
        max_row_idx = i + max_idx_in_bottom 

        c[[i, max_row_idx]] = c[[max_row_idx, i]]
        l[[i, max_row_idx]] = l[[max_row_idx, i]]

        for j in range(i+1, n):
            if(c[i][i]!=0):
                l[j][i] = c[j][i]/c[i][i]
                c[j] = c[j] - c[i]*(c[j][i]/c[i][i])
            # print("forming l: ", l)
    for i in range(n):
        l[i][i]=1
    return c, l

In [59]:
c , l= LU_decomposition(A,b)

In [60]:
L = l
U = c[:, :-1]
b_new = c[:, -1].reshape(-1,1) 
b_new


array([[3138.07531],
       [ 376.56904],
       [3138.07531],
       [ 941.42259]])

In [65]:
print(l)
np.set_printoptions(precision=2, suppress=True) # for getting only 2 decimal places

print(U)
print(b_new)
print(A)

[[ 1.    0.    0.    0.  ]
 [ 0.08  1.    0.    0.  ]
 [ 0.   -8.33  1.    0.  ]
 [ 0.9  -5.83  0.    1.  ]]
[[ 1.    1.    0.    0.  ]
 [ 0.    0.12  0.    0.  ]
 [ 0.   -0.    1.    1.  ]
 [ 0.    0.    0.    0.  ]]
[[3138.08]
 [ 376.57]
 [3138.08]
 [ 941.42]]
[[ 1.    1.    0.    0.  ]
 [ 0.   -1.    1.    1.  ]
 [ 0.9   0.2   0.    0.  ]
 [ 0.08  0.2   0.    0.  ]]


In [62]:
# Back Substitutiom

n = len(b)
x = np.zeros(n)
for row in range(n-1, -1, -1): 
    sum_ax = np.dot(U[row, row+1:], x[row+1:])
    x[row] = (b_new[row][0] - sum_ax) / U[row, row]
    print("Back substituting got x row", row, ":", x[row])
x = x
# x.shape, x, len(x), x[0][0]


Back substituting got x row 3 : inf
Back substituting got x row 2 : -inf
Back substituting got x row 1 : nan
Back substituting got x row 0 : nan


C:\Users\xabc0\AppData\Local\Temp\ipykernel_89820\2206534415.py:7: RuntimeWarning: divide by zero encountered in scalar divide
  x[row] = (b_new[row][0] - sum_ax) / U[row, row]
C:\Users\xabc0\AppData\Local\Temp\ipykernel_89820\2206534415.py:6: RuntimeWarning: invalid value encountered in dot
  sum_ax = np.dot(U[row, row+1:], x[row+1:])
